In [29]:
import networkx as nx

from joblib import Parallel, delayed
from copy import deepcopy

import os

# Comparer au moins deux algorithmes sujet des cours à chaque fois
# (Influence Diffusion , Community Detection, Label Propagation, Graph Embeddings)
# Sur Deux datasets différents !import numpy as np


In [2]:
from sklearn.semi_supervised import LabelPropagation

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Import Data

In [4]:
data_path = '../data/facebook/facebook_combined.txt'
graph = nx.read_edgelist(data_path)

print(f'Graph with {len(graph.nodes)} nodes and {len(graph.edges)} edges was correctly loaded')

Graph with 4039 nodes and 88234 edges was correctly loaded


# Network characteristics

In [5]:
print('Graph main characteristics :')

print(f'Nodes : {len(graph.nodes)}')
print(f'Edges : {len(graph.edges)}')
print(f'Diameter : {nx.diameter(graph)}')

Graph main characteristics :
Nodes : 4039
Edges : 88234
Diameter : 8


# Metrics

In [6]:
metrics_to_functions = {
    'degree': lambda graph : graph.degree, 
    'eigenvector_centrality': nx.eigenvector_centrality, 
    'page_rank': nx.pagerank,
    'clustering_coef': nx.clustering, 
    'closeness': nx.closeness_centrality, 
    'betweenness' : nx.betweenness_centrality
    }
## neighborhood_connectivity missing

def compute_centrality(metric, graph):
    print(f'Metric {metric} is being measured')
    return {metric: metrics_to_functions[metric](graph)}

def get_centralities(graph, metrics_to_functions):
    metric_list = Parallel(n_jobs=4)(delayed(compute_centrality)(metric, graph) for metric in metrics_to_functions.keys())
    metric_dict = {}
    for item in metric_list:
        key = list(item.keys())[0]
        metric_dict[key] = dict(item[key])
    return metric_dict

In [7]:
graph_metrics = get_centralities(graph, metrics_to_functions)

In [8]:
print(graph_metrics.keys())

dict_keys(['degree', 'eigenvector_centrality', 'page_rank', 'clustering_coef', 'closeness', 'betweenness'])


In [9]:
def get_n_maxima_for_metric(n, metric, graph_metrics) :
    temp = graph_metrics[metric].copy()
    max_keys = []
    for i in range(n):
        key = max(temp, key=temp.get)
        max_keys.append(key)
        temp[key] = 0
    return max_keys

max_degree_nodes = get_n_maxima_for_metric(10, "degree", graph_metrics)
max_eigenvector_centrality_nodes = get_n_maxima_for_metric(10, "eigenvector_centrality", graph_metrics)
max_page_rank_nodes = get_n_maxima_for_metric(10, "page_rank", graph_metrics)
max_clustering_coef_nodes = get_n_maxima_for_metric(10, "clustering_coef", graph_metrics)
max_closeness_nodes = get_n_maxima_for_metric(10, "closeness", graph_metrics)
max_betweenness_nodes = get_n_maxima_for_metric(10, "betweenness", graph_metrics)

print(f"Maximum degree nodes : {max_degree_nodes}")
print(f"Maximum eigenvector_centrality nodes : {max_eigenvector_centrality_nodes}")
print(f"Maximum page_rank nodes : {max_page_rank_nodes}")
print(f"Maximum clustering_coef nodes : {max_clustering_coef_nodes}")
print(f"Maximum closeness nodes : {max_closeness_nodes}")
print(f"Maximum betweenness nodes : {max_betweenness_nodes}")

Maximum degree nodes : ['107', '1684', '1912', '3437', '0', '2543', '2347', '1888', '1800', '1663']
Maximum eigenvector_centrality nodes : ['1912', '2266', '2206', '2233', '2464', '2142', '2218', '2078', '2123', '1993']
Maximum page_rank nodes : ['3437', '107', '1684', '0', '1912', '348', '686', '3980', '414', '698']
Maximum clustering_coef nodes : ['32', '33', '35', '42', '44', '46', '47', '52', '63', '70']
Maximum closeness nodes : ['107', '58', '428', '563', '1684', '171', '348', '483', '414', '376']
Maximum betweenness nodes : ['107', '1684', '3437', '1912', '1085', '0', '698', '567', '58', '428']


# Propagation d'une rumeur

## 1. Par random walk

In [10]:
def label_propagation_rw(graph, labeled_nodes, max_iter=2):
    """
    Label propagation using the random walk method.
    """
    # Create a dictionary of node indices to their corresponding row indices in the transition matrix.
    node_to_row = {n: i for i, n in enumerate(graph.nodes())}

    # Create transition matrix
    adjacency_matrix = nx.to_numpy_array(graph)
    row_sums = adjacency_matrix.sum(axis=1)
    transition_matrix = adjacency_matrix / row_sums[:, np.newaxis]

    # Initialize the label matrix.
    Y = np.zeros((len(graph.nodes()), len(labeled_nodes)))
    for i, (node, label) in enumerate(labeled_nodes.items()):
        Y[node_to_row[node], i] = label

    # Propagate the labels using the transition matrix.
    for i in range(max_iter):
        Y_new = transition_matrix.dot(Y)
        if np.allclose(Y, Y_new):
            break
        Y = Y_new

    # Create a dictionary of node indices to their labels.
    labels = {node: np.argmax(Y[node_to_row[node]]) for node in graph.nodes()}

    return labels

In [11]:
node_0 = max_eigenvector_centrality_nodes[0]
node_1 = max_eigenvector_centrality_nodes[1]
labels = label_propagation_rw(graph, {node_0: 0, node_1: 1})

In [12]:
np.sum(list(labels.values()))

766

## 2. Par Supervised learning

In [28]:
def label_propagation_sl(graph, labeled_nodes, max_iter=100):
    """
    Label propagation using a supervised learning approach.
    """
    # Create the feature matrix X and label vector y.
    X = np.array([graph.degree[node] for node in graph.nodes()]).reshape(-1, 1)
    y = np.array([labeled_nodes.get(node, -1) for node in graph.nodes()])

    # Propagate the labels using the label propagation algorithm.
    for i in range(max_iter):
        lp = LabelPropagation(kernel='knn', n_neighbors=5)
        lp.fit(X, y)
        y_new = lp.predict(X)
        if np.allclose(y, y_new):
            break
        y = y_new

    # Create a dictionary of node indices to their labels.
    labels = {node: lp.transduction_[i] for i, node in enumerate(graph.nodes())}

    return labels

In [29]:
node_0 = max_eigenvector_centrality_nodes[0]
node_1 = max_eigenvector_centrality_nodes[1]
labels = label_propagation_sl(graph, {node_0: 0, node_1: 1})

/Users/tanguyblervacque/opt/anaconda3/lib/python3.8/site-packages/sklearn/semi_supervised/_label_propagation.py:231: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer


In [30]:
np.sum(list(labels.values()))

21

In [30]:
graph = nx.read_edgelist('../data/facebook/facebook_combined.txt.log')
print(f"There are {len(graph.nodes)} nodes in graph.")
print(f"There are {len(graph.edges)} edges in graph.")

# print(nx.diameter(graph))


## Different Centralities

# print(graph.degree)
# print(nx.eigenvector_centrality(graph))
# print(nx.pagerank(graph))
# print(nx.clustering(graph))
# print(nx.closeness_centrality(graph)) ## not same formula as in course
# print(nx.betweenness_centrality(graph))


There are 4039 nodes in graph.
There are 88234 edges in graph.


In [31]:
metrics_functions = {'degree': lambda graph : graph.degree, 'eigenvector_centrality': nx.eigenvector_centrality,  
                     'page_rank': nx.pagerank, 'clustering_coef': nx.clustering, 'closeness': nx.closeness_centrality, 
                     'betweenness' : nx.betweenness_centrality }  ## neighborhood_connectivity missing

def compute_centrality(graph, metric, func):   
    return {metric : func(graph)} 


def get_centralities(graph, metrics_functions):
    metrics = {}

    res = Parallel(n_jobs=4)(delayed(compute_centrality)(graph, metric, func) for metric, func in metrics_functions.items())
    
    for dic in res:
        k = list(dic.keys())[0]
        metrics[k] = dict(dic[k])

    return metrics
    

graph_metrics = get_centralities(graph, metrics_functions)

In [32]:
print("We tested these centralities :", list(graph_metrics.keys()), "\n")

print("Overview :")
for v in graph_metrics.values():
    print(v)

We tested these centralities : ['degree', 'eigenvector_centrality', 'page_rank', 'clustering_coef', 'closeness', 'betweenness'] 

Overview :
{'0': 347, '1': 17, '2': 10, '3': 17, '4': 10, '5': 13, '6': 6, '7': 20, '8': 8, '9': 57, '10': 10, '11': 1, '12': 1, '13': 31, '14': 15, '15': 1, '16': 9, '17': 13, '18': 1, '19': 16, '20': 15, '21': 65, '22': 11, '23': 17, '24': 16, '25': 69, '26': 68, '27': 5, '28': 13, '29': 13, '30': 17, '31': 23, '32': 6, '33': 2, '34': 5, '35': 2, '36': 11, '37': 1, '38': 9, '39': 15, '40': 44, '41': 24, '42': 2, '43': 1, '44': 6, '45': 12, '46': 5, '47': 2, '48': 22, '49': 4, '50': 11, '51': 7, '52': 2, '53': 31, '54': 8, '55': 17, '56': 78, '57': 15, '58': 12, '59': 19, '60': 8, '61': 3, '62': 26, '63': 6, '64': 7, '65': 12, '66': 15, '67': 76, '68': 9, '69': 10, '70': 2, '71': 3, '72': 24, '73': 10, '74': 1, '75': 14, '76': 3, '77': 6, '78': 9, '79': 12, '80': 23, '81': 3, '82': 34, '83': 7, '84': 13, '85': 14, '86': 6, '87': 13, '88': 20, '89': 8, '90':

In [33]:
def get_n_maxima_for_metric(n, metric, graph_metrics) :
    temp = deepcopy(graph_metrics[metric])
    max_keys = []
    for i in range(n):
        key = max(temp, key=temp.get)
        max_keys.append(key)
        temp[key] = 0
    return max_keys

get_n_maxima_for_metric(10, "degree", graph_metrics)

['107', '1684', '1912', '3437', '0', '2543', '2347', '1888', '1800', '1663']

In [23]:
graph_metrics["degree"]["107"]

1045